In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
import keras
print(keras.__version__)

2.0.8


In [3]:
image_simulator_dir='data/simulator/'
image_train_dir='data/simulator/train'
image_validation_dir='data/simulator/validation'

## red directory
image_red_light_dir='red/'
image_red_dir = image_simulator_dir + image_red_light_dir

## green directory
image_green_light_dir='green/'
image_green_dir = image_simulator_dir + image_green_light_dir

## yellow directory
image_yellow_light_dir='yellow/'
image_yellow_dir = image_simulator_dir + image_yellow_light_dir

## Generate more training dataset

In [4]:
datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


def augmentated_image(_number_of_outputs, _image_path, _source_image, _image_train_dir, _save_prefix, _save_format):
    img = load_img(_image_path + _source_image)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=_image_train_dir, 
                              save_prefix=_save_prefix, save_format=_save_format):
        i += 1
        if i > _number_of_outputs:
            break  # otherwise the generator would loop indefinitely

### Generate red light training dataset

In [5]:
augmentated_image(200, image_red_dir, 'red_0.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_1.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_2.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_3.jpg', image_train_dir + '/red', 'red', 'jpeg')

### Generate green light training dataset 

In [6]:
augmentated_image(200, image_green_dir, 'green_0.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_1.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_2.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_3.jpg', image_train_dir + '/green', 'green', 'jpeg')

### Generate yellow light training dataset

In [7]:
augmentated_image(200, image_yellow_dir, 'yellow_0.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_1.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_2.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_3.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')

## Validation Data Set

### Generate red light validation dataset

In [8]:
augmentated_image(50, image_red_dir, 'red_0.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_1.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_2.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_3.jpg', image_validation_dir + '/red', 'red', 'jpeg')

### Generate green light validation dataset

In [9]:
augmentated_image(50, image_green_dir, 'green_0.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_1.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_2.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_3.jpg', image_validation_dir + '/green', 'green', 'jpeg')

### Generate yellow light validation dataset

In [10]:
augmentated_image(50, image_yellow_dir, 'yellow_0.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_1.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_2.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_3.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')

## Convolution Neural Network

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [13]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/simulator/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 11450 images belonging to 3 classes.


In [15]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/simulator/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 3490 images belonging to 3 classes.


In [16]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('simulator_traffic_light_model.h5') # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 10s - loss: -4.3347 - acc: 0.5620 - val_loss: -5.4897 - val_acc: 0.6550
Epoch 2/50
125/125 [==============================] - 9s - loss: -5.2913 - acc: 0.6605 - val_loss: -5.2188 - val_acc: 0.6713
Epoch 3/50
125/125 [==============================] - 9s - loss: -5.3676 - acc: 0.6570 - val_loss: -5.3008 - val_acc: 0.6675
Epoch 4/50
125/125 [==============================] - 9s - loss: -5.1660 - acc: 0.6715 - val_loss: -5.1924 - val_acc: 0.6743
Epoch 5/50
125/125 [==============================] - 9s - loss: -5.4592 - acc: 0.6545 - val_loss: -4.9421 - val_acc: 0.6900
Epoch 6/50
125/125 [==============================] - 9s - loss: -5.1156 - acc: 0.6764 - val_loss: -5.4403 - val_acc: 0.6587
Epoch 7/50
125/125 [==============================] - 9s - loss: -5.2121 - acc: 0.6695 - val_loss: -5.4003 - val_acc: 0.6613
Epoch 8/50
125/125 [==============================] - 9s - loss: -5.5747 - acc: 0.6500 - val_loss: -5.3633 - val_acc: 0.6616